In [2]:
import torch
import argparse
import torch.optim
import torch.nn as nn
import torch.utils.data
import torchvision.models as models
import torch.backends.cudnn as cudnn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

import torchvision

In [3]:
import os
import shutil
import argparse
import pandas as pd

In [63]:
df = pd.read_csv('/data/imagenet/LOC_val_solution.csv')

In [64]:
df['class'] = pd.read_csv('/data/imagenet/LOC_val_solution.csv').PredictionString.apply(lambda x: x.split(' ')[0])

In [76]:
df

,ImageId,PredictionString,class
0,ILSVRC2012_val_00048981,n03995372 85 1 499 272,n03995372
1,ILSVRC2012_val_00037956,n03481172 131 0 499 254,n03481172
2,ILSVRC2012_val_00026161,n02108000 38 0 464 280,n02108000
3,ILSVRC2012_val_00026171,n03109150 0 14 216 299,n03109150
4,ILSVRC2012_val_00008726,n02119789 255 142 454 329 n02119789 44 21 322 ...,n02119789
...,...,...,...
49995,ILSVRC2012_val_00005961,n03388043 103 0 279 472,n03388043
49996,ILSVRC2012_val_00008801,n03089624 101 286 170 374 n03089624 236 282 30...,n03089624
49997,ILSVRC2012_val_00008176,n01518878 82 98 439 498,n01518878
49998,ILSVRC2012_val_00004764,n03874293 91 111 490 420,n03874293


In [83]:
data_dir = '/data/imagenet/ILSVRC/Data/CLS-LOC/val/'

for i in range(len(df)):
    label = df['class'][i]
    
    if not os.path.exists(os.path.join(data_dir, label)):
        os.makedirs(os.path.join(data_dir, label))
        
    else:
        shutil.move(os.path.join(data_dir, df['ImageId'][i])+".JPEG", os.path.join(data_dir, label, df['ImageId'][i])+".JPEG")
        

In [4]:
train_dir = '/data/imagenet-100/train'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

train_dataset = ImageFolder(
        train_dir,
        transform=transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))

In [5]:
train_dataset[0]

(tensor([[[ 0.3994,  0.3994,  0.4166,  ...,  0.7419,  0.7933,  0.8276],
          [ 0.4166,  0.4166,  0.4166,  ...,  0.7762,  0.7933,  0.7933],
          [ 0.4337,  0.4337,  0.4337,  ...,  0.8104,  0.7762,  0.7419],
          ...,
          [-0.1486, -0.1486, -0.1657,  ..., -0.0458, -0.0629, -0.0801],
          [-0.2513, -0.2513, -0.2684,  ...,  0.0227, -0.0287, -0.0629],
          [-0.3198, -0.3198, -0.3369,  ...,  0.0569, -0.0116, -0.0629]],
 
         [[ 0.3803,  0.3803,  0.3978,  ...,  0.5203,  0.6078,  0.6604],
          [ 0.3978,  0.3978,  0.3978,  ...,  0.5728,  0.6078,  0.6254],
          [ 0.4153,  0.4153,  0.4153,  ...,  0.6429,  0.6078,  0.5728],
          ...,
          [-0.1450, -0.1450, -0.1450,  ..., -0.8102, -0.8277, -0.8452],
          [-0.2325, -0.2325, -0.2500,  ..., -0.7402, -0.7927, -0.8277],
          [-0.3025, -0.3025, -0.3200,  ..., -0.7052, -0.7752, -0.8277]],
 
         [[ 0.3045,  0.3045,  0.3219,  ...,  0.3742,  0.4439,  0.4962],
          [ 0.3219,  0.3219,

In [101]:
data_train = ImageFolderInstance(train_dir, transform=transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]), two_crop = True)

In [106]:
data_train[0][0].shape

torch.Size([6, 224, 224])

In [95]:
class ImageFolderInstance(datasets.ImageFolder):
    """Folder datasets which returns the index of the image as well
    """

    def __init__(self, root, transform=None, target_transform=None, two_crop=False):
        super(ImageFolderInstance, self).__init__(root, transform, target_transform)
        self.two_crop = two_crop

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target, index) where target is class_index of the target class.
        """
        path, target = self.imgs[index]
        image = self.loader(path)
        if self.transform is not None:
            img = self.transform(image)
        if self.target_transform is not None:
            target = self.target_transform(target)

        if self.two_crop:
            img2 = self.transform(image)
            img = torch.cat([img, img2], dim=0)

        return img, target, index

In [89]:
len(train_dataset)

126689

In [91]:
val_dir = '/data/imagenet-100/val'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

val_dataset = ImageFolder(
        val_dir,
        transform=transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))

In [92]:
len(val_dataset)

4906

In [ ]:
--source_folder: specify the ImageNet-1K data folder (e.g., /root/data/imagenet/train)
--target_folder: specify the ImageNet-100 data folder (e.g., /root/data/imagenet-100/train)
--target_class: specify the ImageNet-100 txt file with list of classes [default: 'IN100.txt']


In [84]:
source_folder = '/data/imagenet/ILSVRC/Data/CLS-LOC/val/'
target_folder = '/data/imagenet-100/val'
target_class = 'imagenet100.txt'

In [85]:
f = []


In [86]:
def generate_data(source_folder, target_folder, target_class):

    txt_data = open(target_class, "r") 
    for ids, txt in enumerate(txt_data):
        s = str(txt.split('\n')[0])
        f.append(s)

    for ids, dirs in enumerate(os.listdir(source_folder)):
        for tg_class in f:
            if dirs == tg_class:
                print('{} is transferred'.format(dirs))
                shutil.copytree(os.path.join(source_folder,dirs), os.path.join(target_folder,dirs)) 



In [87]:
generate_data(source_folder, target_folder, target_class)

n01980166 is transferred
n01735189 is transferred
n02259212 is transferred
n02018207 is transferred
n02107142 is transferred
n01983481 is transferred
n02087046 is transferred
n01820546 is transferred
n04238763 is transferred
n02093428 is transferred
n02116738 is transferred
n02104029 is transferred
n04229816 is transferred
n03837869 is transferred
n07836838 is transferred
n07715103 is transferred
n04435653 is transferred
n02231487 is transferred
n01692333 is transferred
n04111531 is transferred
n02108089 is transferred
n13040303 is transferred
n01729322 is transferred
n04026417 is transferred
n03642806 is transferred
n02974003 is transferred
n03764736 is transferred
n02113799 is transferred
n02009229 is transferred
n02123045 is transferred
n02396427 is transferred
n02086240 is transferred
n01978455 is transferred
n07714571 is transferred
n04067472 is transferred
n03787032 is transferred
n03637318 is transferred
n02100583 is transferred
n03379051 is transferred
n01558993 is transferred


In [9]:
import argparse
import os
import pandas
import shutil
import sys

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm

import utils
from model import Model, Image_Model

In [10]:
import utils

In [11]:
batch_size = 128
train_data, memory_data, test_data = utils.get_dataset('imagenet')

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=24, pin_memory=True, drop_last=True)
memory_loader = DataLoader(memory_data, batch_size=batch_size, shuffle=False, num_workers=24, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=24, pin_memory=True)



In [13]:
memory_loader.dataset.classes

['n01558993',
 'n01692333',
 'n01729322',
 'n01735189',
 'n01749939',
 'n01773797',
 'n01820546',
 'n01855672',
 'n01978455',
 'n01980166',
 'n01983481',
 'n02009229',
 'n02018207',
 'n02085620',
 'n02086240',
 'n02086910',
 'n02087046',
 'n02089867',
 'n02089973',
 'n02090622',
 'n02091831',
 'n02093428',
 'n02099849',
 'n02100583',
 'n02104029',
 'n02105505',
 'n02106550',
 'n02107142',
 'n02108089',
 'n02109047',
 'n02113799',
 'n02113978',
 'n02114855',
 'n02116738',
 'n02119022',
 'n02123045',
 'n02138441',
 'n02172182',
 'n02231487',
 'n02259212',
 'n02326432',
 'n02396427',
 'n02483362',
 'n02488291',
 'n02701002',
 'n02788148',
 'n02804414',
 'n02859443',
 'n02869837',
 'n02877765',
 'n02974003',
 'n03017168',
 'n03032252',
 'n03062245',
 'n03085013',
 'n03259280',
 'n03379051',
 'n03424325',
 'n03492542',
 'n03494278',
 'n03530642',
 'n03584829',
 'n03594734',
 'n03637318',
 'n03642806',
 'n03764736',
 'n03775546',
 'n03777754',
 'n03785016',
 'n03787032',
 'n03794056',
 'n038

In [114]:
model_d = Model(256).cuda()
model_d = nn.DataParallel(model_d)

#     model_path = '../results/cifar10/wcl/no_orient/cifar10_model_d_256_0.1_0.0001_475.pth'
#     state_dict = torch.load(model_path)
#     model_d.load_state_dict(state_dict)

model_b = Model(256).cuda()
model_b = nn.DataParallel(model_b)

In [115]:
x1, x2, tar = iter(train_loader).next()

In [116]:
x1, x2 = x1.cuda(), x2.cuda()

In [118]:
x1.shape

torch.Size([128, 3, 224, 224])

In [1]:
data_tra

NameError: name 'data_tra' is not defined

In [117]:
model_d(x1)

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/taeuk/anaconda3/envs/main/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/taeuk/anaconda3/envs/main/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/taeuk/network/git/HCL/image/model.py", line 25, in forward
    x = self.f(x)
  File "/home/taeuk/anaconda3/envs/main/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/taeuk/anaconda3/envs/main/lib/python3.9/site-packages/torch/nn/modules/container.py", line 141, in forward
    input = module(input)
  File "/home/taeuk/anaconda3/envs/main/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  